# Binning (Diskritisasi) menggunakan K-Means Clustering  

## Clusterisasi dengan K-Means  
Clusterisasi dengan K-Means digunakan untuk mengelompokkan data pada fitur Sepal Length menjadi 4 kelompok (klaster) yaitu 0, 1, 2, 3, 4 berdasarkan tingkat kemiripan nilai. Proses ini dilakukan dengan cara mengelompokkan data yang memiliki nilai panjang sepal yang saling berdekatan ke dalam satu klaster yang sama. Nantinya setiap klaster yang terbentuk akan mewakili satu interval nilai.

In [1]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, accuracy_score
import pandas as pd

# Baca data fitur dan label
df_features = pd.read_excel("data_iris.xlsx")   
df_class = pd.read_excel("class.xlsx")          

# Gabungkan data fitur dan label
df = df_features.copy()
df['class'] = df_class['class']

# Ambil hanya kolom sepal_length
features = df[['sepal_length']]

# Normalisasi fitur
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Clustering KMeans dengan 4 klaster
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
kmeans.fit(scaled_features)

# Simpan hasil cluster ke dataframe
df['cluster'] = kmeans.labels_

# Evaluasi
print(f"Jumlah iterasi sampai konvergen: {kmeans.n_iter_}")
print(f"Inertia (SSE): {kmeans.inertia_:.4f}")
sil_score = silhouette_score(scaled_features, kmeans.labels_)
print(f"Silhouette Score: {sil_score:.4f}")

# Pemetaan cluster ke class mayoritas
mapping = (
    df.groupby('cluster')['class']
    .agg(lambda x: x.mode()[0])
    .to_dict()
)
df['predicted_class'] = df['cluster'].map(mapping)

# Hitung akurasi prediksi clustering terhadap label asli
y_true = df['class']
y_pred = df['predicted_class']
acc = accuracy_score(y_true, y_pred)
print(f"\nAkurasi keseluruhan clustering terhadap label asli: {acc:.4%}")

# Tampilkan distribusi cluster per kelas
dist = pd.crosstab(df['class'], df['cluster'], rownames=['Class'], colnames=['Cluster'])
print("\nDistribusi cluster per kelas:")
print(dist)

# Simpan hasil ke Excel
df.to_excel("clus_dis.xlsx", index=False)

# Jika ingin tampilkan semua hasil
pd.set_option('display.max_rows', None)
print(df[['class', 'cluster', 'predicted_class']])


Jumlah iterasi sampai konvergen: 2
Inertia (SSE): 0.6416
Silhouette Score: 0.5920

Akurasi keseluruhan clustering terhadap label asli: 72.0000%

Distribusi cluster per kelas:
Cluster           0   1   2   3
Class                          
Iris-setosa      10   0  40   0
Iris-versicolor  29   0   5  16
Iris-virginica   10  12   1  27
               class  cluster  predicted_class
0        Iris-setosa        2      Iris-setosa
1        Iris-setosa        2      Iris-setosa
2        Iris-setosa        2      Iris-setosa
3        Iris-setosa        2      Iris-setosa
4        Iris-setosa        2      Iris-setosa
5        Iris-setosa        0  Iris-versicolor
6        Iris-setosa        2      Iris-setosa
7        Iris-setosa        2      Iris-setosa
8        Iris-setosa        2      Iris-setosa
9        Iris-setosa        2      Iris-setosa
10       Iris-setosa        0  Iris-versicolor
11       Iris-setosa        2      Iris-setosa
12       Iris-setosa        2      Iris-setosa
13     

## Cari Min Max dan Centroid dari fitur sepal length  
Pada tahap ini diperoleh informasi statistik yang mencakup nilai minimum (min), maksimum (max), dan centroid (nilai rata-rata) dari masing-masing cluster yang terbentuk. Statistik min dan max yang diperoleh dari hasil clustering ini nantinya dapat digunakan sebagai batas interval dalam proses diskritisasi fitur Sepal Length, sedangkan centroid dapat dimanfaatkan sebagai representasi numerik atau label diskrit dari masing-masing interval. Setiap data Sepal Length yang berada dalam suatu rentang (min hingga max) akan diberi label sesuai klaster tempatnya berada, menjadikan fitur tersebut tidak lagi berbentuk kontinu, melainkan sudah dalam bentuk kategori.


In [23]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Contoh Data
df = pd.read_excel("clus_dis.xlsx")
features = df[['sepal_length']]

# Normalisasi
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# KMeans Clustering
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
kmeans.fit(scaled_features)

# Tambahkan hasil cluster ke data
df['cluster'] = kmeans.labels_

# Ambil centroid dari hasil clustering (dalam skala normalisasi)
centroids_scaled = kmeans.cluster_centers_

# Konversi centroid ke skala asli
centroids_original = scaler.inverse_transform(centroids_scaled)

# Hitung min, max, dan centroid per klaster
cluster_stats = df.groupby('cluster')['sepal_length'].agg(['min', 'max']).copy()
cluster_stats['centroid'] = centroids_original.flatten()

# Tampilkan hasil
print("\nStatistik Sepal Length per Cluster:")
print(cluster_stats)


Statistik Sepal Length per Cluster:
         min  max  centroid
cluster                    
0        5.4  6.1  5.734694
1        7.1  7.9  7.475000
2        4.3  5.3  4.895652
3        6.2  7.0  6.525581


Output tersebut adalah statistik dari fitur Sepal Length yang telah dikelompokkan ke dalam 4 klaster menggunakan KMeans Clustering. Untuk masing-masing klaster ditampilkan:  

* min: Nilai terkecil dari Sepal Length dalam klaster tersebut. Bisa digunakan sebagai batas bawah interval.
* max: Nilai terbesar dari Sepal Length dalam klaster tersebut. Bisa digunakan sebagai batas atas interval.
* centroid: Nilai rata-rata (mean) dari Sepal Length dalam klaster tersebut, yang merupakan pusat dari klaster (hasil centroids_ dari KMeans). 

Statistik min max dapat digunakan sebagai interval untuk diskritisasi pada fitur sepal_length

## Hasil Diskritisasi fitur sepal length  
Pada tahap ini dilakukan proses diskritisasi terhadap fitur numerik sepal_length berdasarkan hasil klasterisasi sebelumnya. Setiap data telah dikelompokkan ke dalam klaster menggunakan algoritma K-Means, dan hasil klaster tersebut kemudian digunakan untuk memberi label diskrit pada nilai sepal_length. Contoh, pada baris pertama, nilai sepal_length_original adalah 5.1 dan termasuk dalam klaster 2 berdasarkan rentang min max, sehingga label diskritisasi menjadi 'C'. Dengan pendekatan ini, fitur sepal_length yang semula berupa nilai kontinu kini telah dikonversi menjadi fitur kategori.

In [3]:
# Pemetaan cluster ke label huruf
cluster_to_label = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D'
}

# Salin kolom sepal_length asli ke kolom baru (agar data numerik tetap tersimpan)
df['sepal_length_original'] = df['sepal_length']

# Gantikan nilai sepal_length dengan huruf berdasarkan klaster
df['sepal_length'] = df['cluster'].map(cluster_to_label)

# Tampilkan hasil
print(df[['cluster', 'sepal_length', 'sepal_length_original']])


     cluster sepal_length  sepal_length_original
0          2            C                    5.1
1          2            C                    4.9
2          2            C                    4.7
3          2            C                    4.6
4          2            C                    5.0
5          0            A                    5.4
6          2            C                    4.6
7          2            C                    5.0
8          2            C                    4.4
9          2            C                    4.9
10         0            A                    5.4
11         2            C                    4.8
12         2            C                    4.8
13         2            C                    4.3
14         0            A                    5.8
15         0            A                    5.7
16         0            A                    5.4
17         2            C                    5.1
18         0            A                    5.7
19         2        

## Mengganti fitur sepal_length(numeriks) ke sepal_length (kategorikal)  
Pada tahap ini, dilakukan proses transformasi fitur sepal_length dari tipe data numerik menjadi kategorikal. Proses ini diawali dengan membaca ulang data iris asli serta data kelas referensi. Data kemudian digabungkan agar fitur sepal_length dapat diproses bersama label kelasnya. Selanjutnya, nilai sepal_length dinormalisasi menggunakan Min-Max Scaler agar seluruh nilai berada dalam rentang 0 hingga 1. Proses normalisasi ini penting untuk meningkatkan performa algoritma K-Means, yang sensitif terhadap skala data.

Setelah dinormalisasi, dilakukan proses klasterisasi menggunakan algoritma K-Means dengan jumlah klaster sebanyak empat. Setiap data kemudian memperoleh label klaster tertentu berdasarkan nilai sepal_length yang dimilikinya. Hasil klasterisasi disimpan dalam kolom baru bernama cluster.Kemudian, nilai klaster tersebut dipetakan ke dalam label kategorikal berupa huruf (‘A’, ‘B’, ‘C’, ‘D’) untuk menggantikan nilai numerik pada fitur sepal_length. Dengan demikian, fitur sepal_length yang semula bertipe numerik kini telah diubah menjadi fitur kategorikal atau diskrit.

In [7]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

df_features = pd.read_excel("clus_dis.xlsx")
df_class = pd.read_excel("class.xlsx")

# Gabungkan dengan class
df = df_features.copy()
df['class'] = df_class['class']

# Clustering ulang fitur sepal_length
features = df[['sepal_length']]
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
kmeans.fit(scaled_features)
df['cluster'] = kmeans.labels_

# Map hasil cluster ke kategori
cluster_to_category = {
    0: 'A',
    2: 'B',
    3: 'C',
    1: 'D'
}
df['sepal_length'] = df['cluster'].map(cluster_to_category)

# Hapus kolom yang tidak perlu
df_result = df.drop(columns=[col for col in ['cluster', 'class', 'predicted_class'] if col in df.columns])

# Simpan
df_result.to_excel("data_iris_sepal_kategori.xlsx", index=False)

# Tampilkan sebagian hasil
pd.set_option('display.max_rows', None)
print(df_result)

      id  petal_length  petal_width sepal_length  sepal_width
0      1           1.4          0.2            B          3.5
1      2           1.4          0.2            B          3.0
2      3           1.3          0.2            B          3.2
3      4           1.5          0.2            B          3.1
4      5           1.4          0.2            B          3.6
5      6           1.7          0.4            A          3.9
6      7           1.4          0.3            B          3.4
7      8           1.5          0.2            B          3.4
8      9           1.4          0.2            B          2.9
9     10           1.5          0.1            B          3.1
10    11           1.5          0.2            A          3.7
11    12           1.6          0.2            B          3.4
12    13           1.4          0.1            B          3.0
13    14           1.1          0.1            B          3.0
14    15           1.2          0.2            A          4.0
15    16

## Melakukan proses diskritisasi dengan K-means clustering pada fitur lainya  
Setelah sebelumnya dilakukan diskritisasi pada fitur sepal_length, tahap ini melanjutkan proses diskritisasi terhadap fitur numerik lainnya, yaitu sepal_width, petal_length, dan petal_width. Metode yang digunakan adalah K-Means Clustering, yang diterapkan secara terpisah pada masing-masing fitur. Pertama, data iris asli dan versi data yang sudah mengandung sepal_length kategorikal dibaca. Kemudian dibuat sebuah fungsi cluster_kategori_stat() yang bertugas melakukan normalisasi menggunakan Min-Max Scaler, proses clustering dengan KMeans, serta menghasilkan kategori huruf berdasarkan label cluster. Fungsi ini juga menghasilkan statistik nilai minimum, maksimum, dan centroid (rata-rata) untuk setiap cluster dari fitur yang didiskritisasi.  

* Fitur sepal_width dikelompokkan ke dalam 3 kategori (‘A’, ‘B’, ‘C’).

* Fitur petal_length dikelompokkan ke dalam 4 kategori (‘A’, ‘B’, ‘C’, ‘D’).

* Fitur petal_width juga dikelompokkan ke dalam 3 kategori (‘A’, ‘B’, ‘C’).

In [12]:
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np # Untuk np.nan jika perlu

# --- 1. Baca data awal ---
# Menggunakan nama file yang telah Anda sediakan
try:
    df_numerik = pd.read_excel("data_iris_sepal_kategori.xlsx")
    # Untuk df_kategori, kita akan memulainya dari df_numerik dan menambahkan kolom kategori.
    # Asumsi: sepal_length sudah ada di df_numerik.
    df_kategori = df_numerik[['id', 'sepal_length']].copy()
except FileNotFoundError as e:
    print(f"Error loading file: {e}. Pastikan file berada di direktori yang sama.")
    exit()

# --- 2. Fungsi bantu diskritisasi + mapping + statistik ---
def discretize_kbins_stat(data_col, n_bins, label_map, strategy='kmeans'):
    """
    Melakukan diskritisasi menggunakan KBinsDiscretizer dan menghitung statistik.

    Args:
        data_col (pd.Series): Kolom data numerik yang akan didiskritisasi.
        n_bins (int): Jumlah bin/kategori yang diinginkan.
        label_map (dict): Kamus untuk memetakan label numerik (0, 1, ...) ke label kategori (A, B, ...).
        strategy (str): Strategi diskritisasi ('uniform', 'quantile', 'kmeans'). Default 'kmeans'.

    Returns:
        tuple: (pd.Series kategori, pd.DataFrame statistik)
    """
    # Inisialisasi KBinsDiscretizer
    # encode='ordinal' berarti outputnya adalah integer (0, 1, ...)
    # strategy='kmeans' akan mencoba membuat bin berdasarkan klaster KMeans
    discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy=strategy)

    # Melakukan fitting dan transformasi
    # Reshape data_col agar sesuai dengan input yang diharapkan oleh scikit-learn (2D array)
    labels_numeric = discretizer.fit_transform(data_col.values.reshape(-1, 1)).flatten().astype(int)

    # Membuat seri kategori huruf
    kategori_series = pd.Series(labels_numeric).map(label_map)

    # Menghitung statistik min, max, centroid per cluster/bin
    df_temp = pd.DataFrame({
        'nilai_asli': data_col,
        'cluster_id': labels_numeric # Menggunakan ID cluster/bin numerik
    })

    # Hitung statistik
    stat = df_temp.groupby('cluster_id')['nilai_asli'].agg(['min', 'max', 'mean'])
    stat = stat.rename(columns={'mean': 'centroid'}) # Mengganti nama 'mean' menjadi 'centroid'

    # Tambahkan kolom kategori ke statistik
    stat['kategori'] = stat.index.map(label_map)
    stat = stat.set_index('kategori')

    return kategori_series, stat

# --- 3. Sepal Width (3 kategori) ---
map_sepal_width = {0: 'A', 1: 'B', 2: 'C'}
# Menggunakan 'kmeans' strategy untuk mendekati perilaku asli
df_kategori['sepal_width'], stat_sepal_width = discretize_kbins_stat(
    df_numerik['sepal_width'], 3, map_sepal_width, strategy='kmeans'
)

# --- 4. Petal Length (4 kategori) ---
map_petal_length = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
df_kategori['petal_length'], stat_petal_length = discretize_kbins_stat(
    df_numerik['petal_length'], 4, map_petal_length, strategy='kmeans'
)

# --- 5. Petal Width (3 kategori) ---
map_petal_width = {0: 'A', 1: 'B', 2: 'C'}
df_kategori['petal_width'], stat_petal_width = discretize_kbins_stat(
    df_numerik['petal_width'], 3, map_petal_width, strategy='kmeans'
)

df_kategori.to_excel("data_iris_kategori_lengkap.xlsx", index=False)
print("Hasil kategori disimpan ke 'data_iris_kategori_lengkap.xlsx'")


# --- 7. Gabungkan semua statistik dan tampilkan ---
print("\n=== Statistik Sepal Width (3 kategori) ===")
print(stat_sepal_width[['min', 'max', 'centroid']])

print("\n=== Statistik Petal Length (4 kategori) ===")
print(stat_petal_length[['min', 'max', 'centroid']])

print("\n=== Statistik Petal Width (3 kategori) ===")
print(stat_petal_width[['min', 'max', 'centroid']])

Hasil kategori disimpan ke 'data_iris_kategori_lengkap.xlsx'

=== Statistik Sepal Width (3 kategori) ===
          min  max  centroid
kategori                    
A         2.0  2.8  2.585106
B         2.9  3.4  3.118987
C         3.5  4.4  3.758333

=== Statistik Petal Length (4 kategori) ===
          min  max  centroid
kategori                    
A         1.0  1.9  1.464000
B         3.0  4.3  3.884000
C         4.4  5.3  4.808889
D         5.4  6.9  5.903333

=== Statistik Petal Width (3 kategori) ===
          min  max  centroid
kategori                    
A         0.1  0.6  0.244000
B         1.0  1.7  1.337037
C         1.8  2.5  2.073913


## Menampilkan semua data hasil diskritisasi setiap fitur

In [ ]:
import pandas as pd

# Baca file Excel yang sudah berisi kategori
df_kategori = pd.read_excel("data_iris_kategori_lengkap.xlsx")

# Tampilkan semua baris
pd.set_option('display.max_rows', None)
print(df_kategori)


      id sepal_length sepal_width petal_length petal_width
0      1            B           C            A           A
1      2            B           B            A           A
2      3            B           B            A           A
3      4            B           B            A           A
4      5            B           C            A           A
5      6            A           C            A           A
6      7            B           B            A           A
7      8            B           B            A           A
8      9            B           B            A           A
9     10            B           B            A           A
10    11            A           C            A           A
11    12            B           B            A           A
12    13            B           B            A           A
13    14            B           B            A           A
14    15            A           C            A           A
15    16            A           C            A          

## Klasifikasi Naive Bayes Data Diskrit

In [14]:
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# Load both files
data_iris = pd.read_excel("data_iris_kategori_lengkap.xlsx")
class_asli = pd.read_excel("class.xlsx")

# Gabungkan data kategorikal dengan kelas asli berdasarkan 'id'
data_gabungan = pd.merge(data_iris, class_asli[['id', 'class']], on='id')

# Encode fitur kategorikal (A, B, dst.) ke numerik
fitur_kategori = ['petal_length', 'petal_width', 'sepal_length', 'sepal_width']
for kolom in fitur_kategori:
    encoder = LabelEncoder()
    data_gabungan[kolom] = encoder.fit_transform(data_gabungan[kolom])

# Encode label kelas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data_gabungan['class'])
X = data_gabungan[fitur_kategori]

# Model Naive Bayes
model = CategoricalNB()
model.fit(X, y)
y_pred = model.predict(X)

# Buat DataFrame hasil prediksi vs kelas asli
hasil_prediksi = data_gabungan[['id']].copy()
hasil_prediksi['kelas_asli'] = label_encoder.inverse_transform(y)
hasil_prediksi['kelas_prediksi'] = label_encoder.inverse_transform(y_pred)

# Simpan ke Excel
output_path = "naive_bayes.xlsx"
hasil_prediksi.to_excel(output_path, index=False)

# Evaluasi hasil prediksi
akurasi = accuracy_score(y, y_pred)
laporan_klasifikasi = classification_report(y, y_pred, target_names=label_encoder.classes_)

# Tampilkan hasil
print("\n=== Hasil Prediksi Kelas ===")
print(hasil_prediksi)

print("\n=== Akurasi ===")
print(f"Akurasi: {akurasi:.2f}")

print("\n=== Laporan Klasifikasi ===")
print(laporan_klasifikasi)



=== Hasil Prediksi Kelas ===
      id       kelas_asli   kelas_prediksi
0      1      Iris-setosa      Iris-setosa
1      2      Iris-setosa      Iris-setosa
2      3      Iris-setosa      Iris-setosa
3      4      Iris-setosa      Iris-setosa
4      5      Iris-setosa      Iris-setosa
5      6      Iris-setosa      Iris-setosa
6      7      Iris-setosa      Iris-setosa
7      8      Iris-setosa      Iris-setosa
8      9      Iris-setosa      Iris-setosa
9     10      Iris-setosa      Iris-setosa
10    11      Iris-setosa      Iris-setosa
11    12      Iris-setosa      Iris-setosa
12    13      Iris-setosa      Iris-setosa
13    14      Iris-setosa      Iris-setosa
14    15      Iris-setosa      Iris-setosa
15    16      Iris-setosa      Iris-setosa
16    17      Iris-setosa      Iris-setosa
17    18      Iris-setosa      Iris-setosa
18    19      Iris-setosa      Iris-setosa
19    20      Iris-setosa      Iris-setosa
20    21      Iris-setosa      Iris-setosa
21    22      Iris-setos

## Klasifikasi Naive Bayes Data Tanpa Diskritisasi

In [ ]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Memuat dataset
try:
    df_iris = pd.read_excel('data_iris.xlsx')
    df_class = pd.read_excel('class.xlsx')
except FileNotFoundError as e:
    print(f"Error loading file: {e}. Pastikan file berada di direktori yang sama.")
    exit()

# Menyiapkan data
# Fitur (X) diambil dari df_iris, mengecualikan kolom 'id'
X = df_iris.drop('id', axis=1)

# Label sebenarnya (y_true) diambil dari kolom 'class' di df_class
y_true = df_class['class']

# Memastikan jumlah baris di X dan y_true cocok
if X.shape[0] != y_true.shape[0]:
    print("Error: Jumlah baris pada fitur dan variabel target sebenarnya tidak cocok.")
    exit()

# Menginisialisasi model Gaussian Naive Bayes
model = GaussianNB()

# Melatih model menggunakan fitur dari df_iris dan label sebenarnya dari df_class
model.fit(X, y_true)

# Membuat prediksi pada fitur yang sama
y_pred = model.predict(X)

# Membandingkan prediksi dengan kelas aktual (y_true)
comparison_df = pd.DataFrame({'True Class': y_true, 'Predicted Class': y_pred})
print("\nPerbandingan Kelas Aktual dan Kelas Prediksi:")
print(comparison_df.head())

# Mengevaluasi model
accuracy = accuracy_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred)

print(f"\nAkurasi: {accuracy:.4f}")
print("\nMatriks Konfusi:")
print(conf_matrix)
print("\nLaporan Klasifikasi:")
print(class_report)

comparison_df.to_excel('naive_bayes_classification_results.xlsx', index=False)


Perbandingan Kelas Aktual dan Kelas Prediksi:
    True Class Predicted Class
0  Iris-setosa     Iris-setosa
1  Iris-setosa     Iris-setosa
2  Iris-setosa     Iris-setosa
3  Iris-setosa     Iris-setosa
4  Iris-setosa     Iris-setosa

Akurasi: 0.9600

Matriks Konfusi:
[[50  0  0]
 [ 0 47  3]
 [ 0  3 47]]

Laporan Klasifikasi:
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        50
Iris-versicolor       0.94      0.94      0.94        50
 Iris-virginica       0.94      0.94      0.94        50

       accuracy                           0.96       150
      macro avg       0.96      0.96      0.96       150
   weighted avg       0.96      0.96      0.96       150



## Klasifikasi Decision Tree Data Diskrit

In [20]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # Meskipun tidak digunakan untuk split di sini, ini adalah praktik baik
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

# --- 1. Memuat data yang sudah didiskritisasi ---
try:
    df_discretized = pd.read_excel('data_iris_kategori_lengkap.xlsx')
    df_true_class = pd.read_excel('class.xlsx')
except FileNotFoundError as e:
    print(f"Error loading file: {e}. Pastikan file berada di direktori yang sama.")
    exit()

# --- 2. Menyiapkan data ---
categorical_features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
X_categorical = df_discretized[categorical_features]

# Variabel target (y) dari file kelas aktual
y = df_true_class['class']

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_encoded = encoder.fit_transform(X_categorical)

feature_names = encoder.get_feature_names_out(categorical_features)
X_df_encoded = pd.DataFrame(X_encoded, columns=feature_names)

if X_df_encoded.shape[0] != y.shape[0]:
    print("Error: Jumlah baris pada fitur yang di-encode dan variabel target tidak cocok.")
    exit()

# --- 3. Melatih pengklasifikasi Decision Tree ---
# random_state digunakan untuk reproduksibilitas hasil
model = DecisionTreeClassifier(random_state=42)

# Melatih model
model.fit(X_df_encoded, y)

# --- 4. Membuat prediksi ---
y_pred = model.predict(X_df_encoded)

# --- 5. Mengevaluasi model ---
accuracy = accuracy_score(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)
class_report = classification_report(y, y_pred)

print(f"\nAkurasi Model Decision Tree (Data Diskritisasi): {accuracy:.4f}")
print("\nMatriks Konfusi:")
print(conf_matrix)
print("\nLaporan Klasifikasi:")
print(class_report)

# Secara opsional, simpan perbandingan kelas aktual dan prediksi ke file CSV
comparison_df_dt = pd.DataFrame({'True Class': y, 'Predicted Class': y_pred})
comparison_df_dt.to_excel('decision_tree_discretized_classification_results.xlsx', index=False)


Akurasi Model Decision Tree (Data Diskritisasi): 0.9800

Matriks Konfusi:
[[50  0  0]
 [ 0 49  1]
 [ 0  2 48]]

Laporan Klasifikasi:
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        50
Iris-versicolor       0.96      0.98      0.97        50
 Iris-virginica       0.98      0.96      0.97        50

       accuracy                           0.98       150
      macro avg       0.98      0.98      0.98       150
   weighted avg       0.98      0.98      0.98       150



In [21]:
import pandas as pd
df= pd.read_excel('decision_tree_discretized_classification_results.xlsx')
# Tampilkan semua baris
pd.set_option('display.max_rows', None)
print(df)

          True Class  Predicted Class
0        Iris-setosa      Iris-setosa
1        Iris-setosa      Iris-setosa
2        Iris-setosa      Iris-setosa
3        Iris-setosa      Iris-setosa
4        Iris-setosa      Iris-setosa
5        Iris-setosa      Iris-setosa
6        Iris-setosa      Iris-setosa
7        Iris-setosa      Iris-setosa
8        Iris-setosa      Iris-setosa
9        Iris-setosa      Iris-setosa
10       Iris-setosa      Iris-setosa
11       Iris-setosa      Iris-setosa
12       Iris-setosa      Iris-setosa
13       Iris-setosa      Iris-setosa
14       Iris-setosa      Iris-setosa
15       Iris-setosa      Iris-setosa
16       Iris-setosa      Iris-setosa
17       Iris-setosa      Iris-setosa
18       Iris-setosa      Iris-setosa
19       Iris-setosa      Iris-setosa
20       Iris-setosa      Iris-setosa
21       Iris-setosa      Iris-setosa
22       Iris-setosa      Iris-setosa
23       Iris-setosa      Iris-setosa
24       Iris-setosa      Iris-setosa
25       Iri

## Klasifikasi Decision Tree Data Tanpa Diskritisasi

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # Termasuk untuk praktik baik
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# --- 1. Memuat data Iris asli ---
try:
    df_iris_original = pd.read_excel('data_iris.xlsx')
    df_true_class = pd.read_excel('class.xlsx')
except FileNotFoundError as e:
    print(f"Error loading file: {e}. Pastikan file berada di direktori yang sama.")
    exit()

# --- 2. Menyiapkan data ---
# Fitur (X) dari data Iris asli, mengecualikan kolom 'id'
features = ['petal_length', 'petal_width', 'sepal_length', 'sepal_width']
X = df_iris_original[features]

# Variabel target (y) dari file kelas aktual
y = df_true_class['class']

# Memastikan jumlah sampel di X dan y cocok
if X.shape[0] != y.shape[0]:
    print("Error: Jumlah baris pada fitur dan variabel target tidak cocok.")
    exit()

# Untuk tujuan demonstrasi ini, kita akan melatih dan mengevaluasi pada seluruh dataset.
# Dalam skenario dunia nyata, sangat disarankan untuk menggunakan train_test_split
# untuk membagi data menjadi set pelatihan dan pengujian guna mendapatkan evaluasi model yang lebih realistis.
# Contoh penggunaan train_test_split:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- 3. Melatih pengklasifikasi Decision Tree ---
# random_state digunakan untuk reproduksibilitas hasil
model = DecisionTreeClassifier(random_state=42)

# Melatih model
model.fit(X, y)

# --- 4. Membuat prediksi ---
y_pred = model.predict(X)

# --- 5. Mengevaluasi model ---
accuracy = accuracy_score(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)
class_report = classification_report(y, y_pred)

print(f"\nAkurasi Model Decision Tree (Data Iris Asli): {accuracy:.4f}")
print("\nMatriks Konfusi:")
print(conf_matrix)
print("\nLaporan Klasifikasi:")
print(class_report)

comparison_df_dt_original = pd.DataFrame({'True Class': y, 'Predicted Class': y_pred})
comparison_df_dt_original.to_excel('decision_tree_original_iris_classification_results.xlsx', index=False)


Akurasi Model Decision Tree (Data Iris Asli): 1.0000

Matriks Konfusi:
[[50  0  0]
 [ 0 50  0]
 [ 0  0 50]]

Laporan Klasifikasi:
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        50
Iris-versicolor       1.00      1.00      1.00        50
 Iris-virginica       1.00      1.00      1.00        50

       accuracy                           1.00       150
      macro avg       1.00      1.00      1.00       150
   weighted avg       1.00      1.00      1.00       150


Hasil klasifikasi Decision Tree pada data Iris asli disimpan ke 'decision_tree_original_iris_classification_results.xlsx'


In [ ]:
import pandas as pd
df= pd.read_excel('decision_tree_original_iris_classification_results.xlsx')

# Tampilkan semua baris
pd.set_option('display.max_rows', None)
print(df)

          True Class  Predicted Class
0        Iris-setosa      Iris-setosa
1        Iris-setosa      Iris-setosa
2        Iris-setosa      Iris-setosa
3        Iris-setosa      Iris-setosa
4        Iris-setosa      Iris-setosa
5        Iris-setosa      Iris-setosa
6        Iris-setosa      Iris-setosa
7        Iris-setosa      Iris-setosa
8        Iris-setosa      Iris-setosa
9        Iris-setosa      Iris-setosa
10       Iris-setosa      Iris-setosa
11       Iris-setosa      Iris-setosa
12       Iris-setosa      Iris-setosa
13       Iris-setosa      Iris-setosa
14       Iris-setosa      Iris-setosa
15       Iris-setosa      Iris-setosa
16       Iris-setosa      Iris-setosa
17       Iris-setosa      Iris-setosa
18       Iris-setosa      Iris-setosa
19       Iris-setosa      Iris-setosa
20       Iris-setosa      Iris-setosa
21       Iris-setosa      Iris-setosa
22       Iris-setosa      Iris-setosa
23       Iris-setosa      Iris-setosa
24       Iris-setosa      Iris-setosa
25       Iri

## Kesimpulan  
$
\begin{array}{lcccc}
\hline
\textbf{Model} & \textbf{Akurasi} \\
\hline
\text{Naive Bayes (diskritisasi)} & 0.97 \\
\text{Naive Bayes (iris-asli)} & 0.96 \\
\text{Decision Tree (diskritisasi)} & 0.98 \\
\text{Decision Tree (iris-asli)} & 1.00 \\
\hline
\end{array}
$  

Secara keseluruhan, Decision Tree menunjukkan kinerja yang lebih unggul dibandingkan Naive Bayes pada kedua jenis data.Pengolahan data (diskritisasi) memberikan dampak yang berbeda pada kedua model:

* Pada Naive Bayes, proses diskritisasi justru meningkatkan kinerja model dari 0.96 menjadi 0.97. Ini menunjukkan bahwa model Naive Bayes dalam kasus ini bekerja lebih baik dengan fitur-fitur yang bersifat kategorikal (hasil diskritisasi).
* Pada Decision Tree, proses diskritisasi sedikit menurunkan kinerja model dari 1.00 menjadi 0.98. Hal ini menandakan bahwa Decision Tree mampu memanfaatkan informasi dari data numerik kontinu pada set data Iris asli secara lebih efektif untuk mencapai hasil yang sempurna.
